In [1]:
import numpy as np
import pandas as pd
import re
import time
import io
from datasketch import MinHash, MinHashLSHForest

In [2]:
#Preprocess will split a string of text into individual tokens/shingles based on whitespace.
def preprocess(text):
    text = re.sub(r'[^\w\s]','',text)
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

In [3]:
#Example of pre-processing
text = 'The devil went down to Georgia'
print('The shingles (tokens) are:', preprocess(text))

The shingles (tokens) are: ['the', 'devil', 'went', 'down', 'to', 'georgia']


In [4]:
#choosing parameters
#Number of Permutations
permutations = 128

#Number of Recommendations to return
num_recommendations = 1

In [5]:
#Create Minhash Forest for queries
def get_forest(data, perms):
    start_time = time.time()
    
    minhash = []
    
    for text in data['text']:
        tokens = preprocess(text)
        m = MinHash(num_perm=perms)
        for s in tokens:
            m.update(s.encode('utf8'))
        minhash.append(m)
        
    forest = MinHashLSHForest(num_perm=perms)
    
    for i,m in enumerate(minhash):
        forest.add(i,m)
        
    forest.index()
    
    print('It took %s seconds to build forest.' %(time.time()-start_time))
    
    return forest

In [6]:
#Evaluate Queries
def predict(text, database, perms, num_results, forest):
    start_time = time.time()
    
    tokens = preprocess(text)
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    
    result = database.iloc[idx_array]['title']
    
    print('It took %s seconds to query forest.' %(time.time()-start_time))
    
    return result

In [7]:
db = pd.read_csv("/Users/sachingupta/L_S_H/output/papers_new.csv")
db['text'] = db['title'] + ' ' + db['abstract']
forest = get_forest(db, permutations)

It took 12.202419996261597 seconds to build forest.


In [8]:
num_recommendations = 5
title = 'Using a neural net to instantiate a deformable model'
result = predict(title, db, permutations, num_recommendations, forest)
print('\n Top Recommendation(s) is(are) \n', result)

It took 0.00785374641418457 seconds to query forest.

 Top Recommendation(s) is(are) 
 226     Neural Network Weight Matrix Synthesis Using O...
488     A Self-Organizing Integrated Segmentation and ...
203     Analytic Solutions to the Formation of Feature...
976     Using a neural net to instantiate a deformable...
3165    Inferring Neural Firing Rates from Spike Train...
Name: title, dtype: object
